# Extracting Patents linked to a Research Organization via the Dimensions API
_Last Updated: November 2019_

This tutorial shows how to extract and analyse patents information linked to a selected research organization, using the Dimensions Analytics API and [GRID](https://grid.ac/). 

## Load libraries and log in

In [0]:
username = ""  #@param {type: "string"}
password = ""  #@param {type: "string"}
endpoint = "https://app.dimensions.ai"  #@param {type: "string"}

#
!pip install dimcli plotly_express -U --quiet 
#
# API library
import dimcli
from dimcli.shortcuts import *
dimcli.login(username, password, endpoint)
dsl = dimcli.Dsl()
#
# other libraries
import plotly_express as px
import pandas as pd
from pandas.io.json import json_normalize
import time
import json
from tqdm import tqdm_notebook as progressbar


     |████████████████████████████████| 122kB 2.8MB/s 
DimCli v0.6.1 - Succesfully connected to <https://app.dimensions.ai> (method: manual login)


## Choose a Research Organization

Dimensions uses GRID to identify research-related organizations, eg https://grid.ac/institutes/grid.89170.37 

In [0]:
#@markdown The organization we are extracting data for
GRIDID = "grid.89170.37" #@param {type: "string"} 
#@markdown The start/end year of publications used to extract patents
YEAR_START = 2006 #@param {type: "slider", min: 1950, max: 2020}
YEAR_END = 2016 #@param {type: "slider", min: 1950, max: 2020}

if YEAR_END < YEAR_START:
  YEAR_END = YEAR_START
#@markdown ---

___
## 1 - Extracting Publications data

In [0]:
# Get full list of publications linked to this organization for the selected time frame

q = f"""search publications 
        where research_orgs.id="{GRIDID}" 
        and year in [{YEAR_START}:{YEAR_END}]
        return publications[basics+category_for+times_cited]"""

pubs_json = dsl.query_iterative(q)
pubs = pubs_json.as_dataframe()

# save the data
pubs.to_csv(f"pubs_{GRIDID}.csv")
print(f"Data saved to 'pubs_{GRIDID}.csv'")

1000 / 11164
2000 / 11164
3000 / 11164
4000 / 11164
5000 / 11164
6000 / 11164
7000 / 11164
8000 / 11164
9000 / 11164
10000 / 11164
11000 / 11164
11164 / 11164
Data saved to 'pubs_grid.89170.37.csv'


### How many publications per year?

In [0]:
px.histogram(pubs, x="year", y="id", color="type", barmode="group", title=f"Publication by year from {GRIDID}")

### What are the main publication venues?

In [0]:
# group pubs by journal
journals = pubs.groupby(['journal.title', 'year'], as_index=False).count().sort_values('id', ascending=False)
# plot it
px.bar(journals[:200], x="journal.title", y="id",  color="year", title=f"Top Publication Venues from {GRIDID}")

### What are the main subject areas? 

We can use the Field of Research categories information in publications to obtain a breakdown of the publications by subject areas. 

This can be achieved by 'exploding' the `category_for` data into a separate table, since there can be more than one category per publication. The new categories table also retains some basic info about the publications it relates to eg *journal*, *title*, *publication id* etc.. so to make it easier to analyse the data. 

In [0]:
# ensure the key exists in all rows (even if empty)
normalize_key("category_for", pubs_json.publications)
# explode subjects into separate table
pubs_subjects = json_normalize(pubs_json.publications, record_path=['category_for'], 
                               meta=["id", "type", ["journal", "title"], "year"], 
                               errors='ignore', record_prefix='for_')
# add a new column: category name without digits for better readability 
pubs_subjects['topic'] = pubs_subjects['for_name'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))

Let's take a quick look at the data

In [0]:
pubs_subjects.head(5)

,for_id,for_name,id,type,journal.title,year,topic
0,2202,02 Physical Sciences,pub.1055138629,article,ACS Omega,2016,Physical Sciences
1,2421,0299 Other Physical Sciences,pub.1055138629,article,ACS Omega,2016,Other Physical Sciences
2,2620,0604 Genetics,pub.1062714505,article,Microbiology Resource Announcements,2016,Genetics
3,2206,06 Biological Sciences,pub.1062714505,article,Microbiology Resource Announcements,2016,Biological Sciences
4,3114,1108 Medical Microbiology,pub.1062714433,article,Microbiology Resource Announcements,2016,Medical Microbiology


Now we can build a scatter plot that shows the amount and distribution of categories of the years.

In [0]:
px.scatter(pubs_subjects, x="year", y="topic", color="type", 
           hover_name="for_name", 
           height=800,
           marginal_x="histogram", marginal_y="histogram", 
           title=f"Top publication subjects for {GRIDID} (marginal subplots = X/Y totals)")

Output hidden; open in https://colab.research.google.com to view.

## 2 - Extracting Patents linked to Publications

> Tip: see the API [data model](https://docs.dimensions.ai/dsl/data-model.html) page for more details on which relationships exist between publications, patents and the other Dimensions source types.

In this section we extract all patents linked to the publications dataset previously created. The steps are the following:

* launch a looped patents query using the `publication_ids` field and the IDs from the publications dataset previously extracted
* remove duplicates from patents and save the results 
* count patents per publication and enrich the original publication dataset with these numbers  

In [0]:
#
# the main query
#
q = """search patents where publication_ids in {} 
  return patents[basics+publication_ids+FOR]"""


#
# useful libraries for looping
#
from dimcli.shortcuts import chunks_of
from tqdm import tqdm_notebook as progressbar

#
# let's loop through all grants IDs in chunks and query Dimensions 
#
print("===\nExtracting patents data ...")
patents_json = []
BATCHSIZE = 400
VERBOSE = False # set to True to see patents extraction logs
pubsids = pubs['id']

for chunk in progressbar(list(chunks_of(list(pubsids), 400))):
    data = dsl.query_iterative(q.format(json.dumps(chunk)), verbose=VERBOSE)
    patents_json += data.patents
    time.sleep(1)
    

#
# put the patents data into a dataframe, remove duplicates and save
#
patents = pd.DataFrame().from_dict(patents_json)
print("Patents found: ", len(patents))
patents.drop_duplicates(subset='id', inplace=True)
print("Unique Patents found: ", len(patents))
patents.to_csv("patents.csv", index=False)
# turning lists into strings to ensure compatibility with CSV loaded data
# see also: https://stackoverflow.com/questions/23111990/pandas-dataframe-stored-list-as-string-how-to-convert-back-to-list
patents['publication_ids'] = patents['publication_ids'].apply(lambda x: ','.join(map(str, x))) 
print("===\nDone - data saved as 'patents.csv'")


#
# count patents per publication and enrich the original dataset
#
def patents_per_pub(pubid):
  global patents
  return patents[patents['publication_ids'].str.contains(pubid)]

print("===\nCounting patents per publication...")

l = []
for x in progressbar(pubsids):
  l.append(len(patents_per_pub(x)))

#
# save the data
#
pubs['patents'] = l
# save the data
pubs.to_csv(f"pubs_{GRIDID}_enriched_patents.csv", index=False)
print(f"Data saved to 'pubs_{GRIDID}_enriched_patents.csv'")

===
Extracting patents data ...



Patents found:  1545
Unique Patents found:  1342
===
Done - data saved as 'patents.csv'
===
Counting patents per publication...



Data saved to 'pubs_grid.89170.37_enriched_patents.csv'


### A quick look at the data

In [0]:
# display top 3 rows
patents.head(3)

,publication_ids,id,inventor_names,publication_date,assignee_names,assignees,filing_status,title,year,times_cited,FOR,granted_year
0,"pub.1038200468,pub.1067588797,pub.1006473292,p...",WO-2017153345-A1,"[TAVERNIER, JAN, VAN DER HEYDEN, José, DEVOOGD...",2017-09-14,"[VIB VZW, UNIV GENT, UNIV BRUSSEL VRIJE]","[{'id': 'grid.8767.e', 'acronym': 'VUB', 'name...",Application,CD20 BINDING AGENTS AND USES THEREOF,2017,NaN,NaN,NaN
1,"pub.1021290825,pub.1034244510,pub.1022541167",US-20160240719-A1,"[Meng-Yu Lin, Shih-Yen Lin, Si-Chen Lee, Samue...",2016-08-18,[Taiwan Semiconductor Manufacturing Co (TSMC) ...,"[{'id': 'grid.454156.7', 'acronym': 'TSMC', 'n...",Application,Semiconductor Devices Comprising 2D-Materials ...,2015,11.0,"[{'id': '2921', 'name': '0912 Materials Engine...",NaN
2,"pub.1073831920,pub.1034244510,pub.1021290825,p...",US-9859115-B2,"[Meng-Yu Lin, Shih-Yen Lin, Si-Chen Lee, Samue...",2018-01-02,[Taiwan Semiconductor Manufacturing Co (TSMC) ...,"[{'id': 'grid.454156.7', 'acronym': 'TSMC', 'n...",Grant,Semiconductor devices comprising 2D-materials ...,2015,NaN,"[{'id': '2921', 'name': '0912 Materials Engine...",2018.0


Publications now have patents info:

In [0]:
pubs.sort_values("patents", ascending=False).head(3)

,title,author_affiliations,type,id,times_cited,category_for,pages,year,volume,issue,journal.id,journal.title,patents
6358,Peptides for specific intracellular delivery a...,"[[{'first_name': 'James B', 'last_name': 'Dele...",article,pub.1037665483,64.0,"[{'id': '2921', 'name': '0912 Materials Engine...",411-433,2010,1,3,jour.1044036,Therapeutic Delivery,44
3534,Energy Conversion and Transmission Modules for...,"[[{'first_name': 'Paul', 'last_name': 'Jaffe',...",article,pub.1061297785,35.0,"[{'id': '2921', 'name': '0912 Materials Engine...",1424-1437,2013,101,6,jour.1022875,Proceedings of the IEEE,37
10256,Solution-phase single quantum dot fluorescence...,"[[{'first_name': 'Thomas', 'last_name': 'Pons'...",article,pub.1055843130,184.0,"[{'id': '2447', 'name': '0303 Macromolecular a...",15324-31,2006,128,47,jour.1081898,Journal of the American Chemical Society,36


In [0]:
px.scatter(pubs, x="patents", y="times_cited")

## 3 - Patents Data Analysis

Now that we have extracted all the data we need, let's start exploring them by building a few visualizations. 

###  How many patents per year? 

In [0]:
# PS is year correct as a patents field?
px.histogram(patents, x="year", y="id", color="filing_status", 
             barmode="group", 
             title=f"Patents referencing publications from {GRIDID} - by year")

### Who is filing the patents?

This can be done by looking at the field `assigness` of patent. Since the field contains nested information, first we need to extract it into its own table (similarly to what we've done above with publications categories). 

In [0]:
# ensure the key exists in all rows (even if empty) 
normalize_key('assignees', patents_json)
# explode assigness into separate table 
patents_assignees = json_normalize(patents_json, 
                                   record_path=['assignees'], 
                                   meta=['id', 'year', 'title'], 
                                   meta_prefix="patent_")

In [0]:
patents_assignees.head(2)

,id,acronym,name,country_name,patent_id,patent_year,patent_title
0,grid.8767.e,VUB,Vrije Universiteit Brussel,Belgium,WO-2017153345-A1,2017,CD20 BINDING AGENTS AND USES THEREOF
1,grid.5342.0,NaN,Ghent University,Belgium,WO-2017153345-A1,2017,CD20 BINDING AGENTS AND USES THEREOF


In [0]:
top_assignees = patents_assignees.groupby(['name', 'country_name'],  as_index=False).count().sort_values(by="patent_id", ascending=False)
px.bar(top_assignees, x="name", y="patent_id", 
       hover_name="name", color="country_name",
       title=f"Top Assignees for patents referencing publications from {GRIDID}")

In [0]:
px.scatter(patents_assignees,  x="name", y="country_name", 
           color="patent_year", hover_name="name",  
           hover_data=["id", "patent_id"],  marginal_y="histogram",
           title=f"Assignees for patents referencing publications from {GRIDID} - Yearly breakdown")

### What are the most cited publications?

In [0]:
pubs_cited = pubs.query("patents > 0 ").sort_values('patents', ascending=False).copy()
pubs_cited

,title,author_affiliations,type,id,times_cited,category_for,pages,year,volume,issue,journal.id,journal.title,patents
6358,Peptides for specific intracellular delivery a...,"[[{'first_name': 'James B', 'last_name': 'Dele...",article,pub.1037665483,64.0,"[{'id': '2921', 'name': '0912 Materials Engine...",411-433,2010,1,3,jour.1044036,Therapeutic Delivery,44
3534,Energy Conversion and Transmission Modules for...,"[[{'first_name': 'Paul', 'last_name': 'Jaffe',...",article,pub.1061297785,35.0,"[{'id': '2921', 'name': '0912 Materials Engine...",1424-1437,2013,101,6,jour.1022875,Proceedings of the IEEE,37
10256,Solution-phase single quantum dot fluorescence...,"[[{'first_name': 'Thomas', 'last_name': 'Pons'...",article,pub.1055843130,184.0,"[{'id': '2447', 'name': '0303 Macromolecular a...",15324-31,2006,128,47,jour.1081898,Journal of the American Chemical Society,36
8876,Ultra-low resistivity Al+ implanted 4H–SiC obt...,"[[{'first_name': 'Siddarth G.', 'last_name': '...",article,pub.1005512153,12.0,"[{'id': '2921', 'name': '0912 Materials Engine...",140-145,2008,52,1,jour.1312102,Solid-State Electronics,34
10714,Excimer laser forward transfer of mammalian ce...,"[[{'first_name': 'A.', 'last_name': 'Doraiswam...",article,pub.1036305541,103.0,"[{'id': '2921', 'name': '0912 Materials Engine...",4743-4747,2006,252,13,jour.1038686,Applied Surface Science,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8685,Free-space optical communication link across 1...,"[[{'first_name': 'Mark', 'last_name': 'Plett',...",article,pub.1046523659,18.0,"[{'id': '2208', 'name': '08 Information and Co...",045001-045001-10,2008,47,4,jour.1122763,Optical Engineering,1
5175,Demonstration of submersible high-throughput m...,"[[{'first_name': 'André A', 'last_name': 'Adam...",article,pub.1055001677,18.0,"[{'id': '2203', 'name': '03 Chemical Sciences'...",8411-9,2011,83,22,jour.1345331,Analytical Chemistry,1
8697,Nanoscale control of an interfacial metal–insu...,"[[{'first_name': 'C.', 'last_name': 'Cen', 'co...",article,pub.1004142357,390.0,"[{'id': '2921', 'name': '0912 Materials Engine...",298-302,2008,7,4,jour.1031408,Nature Materials,1
8699,Formation of a new phase of barium copper sulf...,"[[{'first_name': 'J.A.', 'last_name': 'Frantz'...",article,pub.1001945512,2.0,"[{'id': '2921', 'name': '0912 Materials Engine...",1582-1584,2008,62,10-11,jour.1042608,Materials Letters,1


In [0]:
px.bar(pubs_cited[:1000], color="type",
       x="year", y="patents", 
       hover_name="title",  hover_data=["journal.title"],
       title=f"Top Publications from {GRIDID} mentioned in patents, by year of publication")

### What are the main subject areas of cited publications?

In [0]:
citedids = list(pubs_cited[:1000]['id'])
pubs_subjects_cited = pubs_subjects[pubs_subjects['id'].isin(citedids)]

In [0]:
px.scatter(pubs_subjects_cited, x="year", y="topic", color="type", 
           hover_name="for_name", 
           height=800,
           marginal_x="histogram", marginal_y="histogram", 
           title=f"Top {GRIDID} publication subjects cited by patents")

---
## Want to learn more?

Check out the [Dimensions API Lab](https://digital-science.github.io/dimensions-api-lab/) website, which contains many tutorials and reusable Jupyter notebooks for scholarly data analytics. 